In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version

import copy
import itertools
import adjustText

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
from Utilities_df import DFConstructType
import Plot_General
import Plot_Box_sns
import Plot_Hist
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

# Pretty much everything from this notebook has been moved to other classes/modules/whatever

# ===============================
# BEGIN ACQUISITION METHODS
# ===============================

# TODO Need to update build_active_MP_for_outages_df in other notebooks, as the method below is the most recent
Also need to add build_active_MP_for_xfmrs_in_outages_df (below)

In [ ]:
#TODO !!!!!!!!!!!! IMPORTANT !!!!!!!!!!!!!!
# In order for this to function as desired, reduce_active_MP_for_outages must group by transformer pole number
#   for this case, not outage number as in the build_active_MP_for_outages_df case
# Some other adjustment may be needed as well!
# NOTE: There are some instances where build_active_MP_for_outages_df will find PNs not found by 
#         build_active_MP_for_xfmrs_in_outages_df.
#       The reason being that the former doesn't care whether or not the trsf_pole_nb is NaN, but this function does!
# May want to, e.g., add in some sort of dropna option
# However, what would one do in such a situation?  I guess you would have to make an or statement, allowing
#   a PN to have the correct transformer OR be listed in df_outage is partaking in the outage?
# Not too sure about this, but it will obviously need some work.
# For example where build_active_MP_for_outages_df finds additional PN, see outg_rec_nb 13297062, where
#   mfr_devc_ser_nbr=995582422 is found by build_active_MP_for_outages_df but not by build_active_MP_for_xfmrs_in_outages_df
#   (because only found in historic, and therefore the trsf_pole_nb could not be determined!)
#
# May need to combine those with trsf_pole_nb=na in mp to those in mp_df_curr_hist_dict
#   One could, e.g., call MeterPremise.build_mp_df_curr_hist_for_PNs for any PNs without a transformer 
#   pole number, and still use MeterPremise.build_mp_df_curr_hist_for_xfmrs for others
def build_active_MP_for_xfmrs_in_outages_df(
    df_outage, 
    prem_nb_col, 
    df_outage_trsf_pole_nb_col=None, 
    drop_inst_rmvl_cols=False, 
    outg_rec_nb_idfr='OUTG_REC_NB', 
    is_slim=False, 
    addtnl_mp_df_curr_cols=None, 
    addtnl_mp_df_hist_cols=None, 
    dt_on_ts_col='DT_ON_TS', 
    df_off_ts_full_col='DT_OFF_TS_FULL', 
    consolidate_PNs_batch_size=1000, 
    df_mp_serial_number_col='mfr_devc_ser_nbr', 
    df_mp_prem_nb_col='prem_nb', 
    df_mp_install_time_col='inst_ts', 
    df_mp_removal_time_col='rmvl_ts', 
    df_mp_trsf_pole_nb_col='trsf_pole_nb', 
    early_return=False
):
    r"""
    Similar to build_active_MP_for_outages/build_active_MP_for_outages_df
    
    df_outage_trsf_pole_nb_col:
        If trsf_pole_nb already present in df_outage, set this equal to the column.
        Otherwise, the trsf_pole_nbs will be acquired from the current Meter Premise table
        
    If addtnl_mp_df_curr_cols and addtnl_mp_df_hist_cols are included (i.e., are not None), their intersection will be added
      as addtnl_groupby_cols input argument to drop_approx_duplicates (otherwise, they would be returned in the DF as list
      objects, and would likely need to be exploded)
    """
    #-------------------------
    assert(prem_nb_col in df_outage.columns and 
           dt_on_ts_col in df_outage.columns and 
           df_off_ts_full_col in df_outage.columns)
    #-------------------------
    # If trsf_pole_nbs present in df_outage, simply grab.
    # Otherwise, use current Meter Premise table together with PNs from df_outage to acquire trsf_pole_nbs
    if df_outage_trsf_pole_nb_col is not None and df_outage_trsf_pole_nb_col in df_outage.columns:
        trsf_pole_nbs = df_outage[df_outage_trsf_pole_nb_col].unique().tolist()
    else:
        if not is_slim:
            PNs = df_outage[prem_nb_col].unique().tolist()
        else:
            PNs = Utilities_df.consolidate_column_of_lists(
                df=df_outage, 
                col=prem_nb_col, 
                sort=True,
                include_None=False,
                batch_size=consolidate_PNs_batch_size, 
                verbose=False
            )
        #-----
        PNs = [x for x in PNs if pd.notna(x)]
        PNs_type = type(PNs[0])
        assert(Utilities.are_all_list_elements_of_type(PNs, PNs_type))
        #-------------------------
        mp_cols_of_interest=[df_mp_prem_nb_col, df_mp_trsf_pole_nb_col]
        mp = MeterPremise(
            df_construct_type=DFConstructType.kRunSqlQuery, 
            init_df_in_constructor=True, 
            build_sql_function=MeterPremise.build_sql_meter_premise, 
            build_sql_function_kwargs=dict(
                cols_of_interest=mp_cols_of_interest, 
                premise_nbs=PNs, 
                table_name='meter_premise'
            )
        )
        mp_df = mp.df
        #-------------------------
        trsf_pole_nbs = mp_df[df_mp_trsf_pole_nb_col].unique().tolist()
    #--------------------------------------------------
    if addtnl_mp_df_curr_cols is not None and addtnl_mp_df_hist_cols is not None:
        assert(Utilities.is_object_one_of_types(addtnl_mp_df_curr_cols, [list, tuple]))
        assert(Utilities.is_object_one_of_types(addtnl_mp_df_hist_cols, [list, tuple]))
        drop_approx_duplicates_args = dict(
            addtnl_groupby_cols=list(set(addtnl_mp_df_curr_cols).intersection(set(addtnl_mp_df_hist_cols)))
        )
    else:
        drop_approx_duplicates_args=None
    #-----
    mp_df_curr_hist_dict = MeterPremise.build_mp_df_curr_hist_for_xfmrs(
        trsf_pole_nbs=trsf_pole_nbs, 
        join_curr_hist=False, 
        addtnl_mp_df_curr_cols=addtnl_mp_df_curr_cols, 
        addtnl_mp_df_hist_cols=addtnl_mp_df_hist_cols, 
        assume_one_xfmr_per_PN=True, 
        drop_approx_duplicates=True, 
        drop_approx_duplicates_args=drop_approx_duplicates_args, 
        df_mp_serial_number_col=df_mp_serial_number_col, 
        df_mp_prem_nb_col=df_mp_prem_nb_col, 
        df_mp_install_time_col=df_mp_install_time_col, 
        df_mp_removal_time_col=df_mp_removal_time_col, 
        df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
    )
    df_mp_curr = mp_df_curr_hist_dict['mp_df_curr']
    df_mp_hist = mp_df_curr_hist_dict['mp_df_hist']
    if early_return:
        return df_mp_curr, df_mp_hist
    #-------------------------
    active_SNs_df = reduce_active_MP_for_outages(
        df_outage=df_outage, 
        prem_nb_col=prem_nb_col, 
        df_mp_curr=df_mp_curr, 
        df_mp_hist=df_mp_hist, 
        drop_inst_rmvl_cols=drop_inst_rmvl_cols, 
        outg_rec_nb_idfr=outg_rec_nb_idfr, 
        is_slim=is_slim, 
        drop_approx_duplicates=True, 
        drop_approx_duplicates_args=drop_approx_duplicates_args, 
        dt_on_ts_col=dt_on_ts_col, 
        df_off_ts_full_col=df_off_ts_full_col, 
        df_mp_serial_number_col=df_mp_serial_number_col, 
        df_mp_prem_nb_col=df_mp_prem_nb_col, 
        df_mp_install_time_col=df_mp_install_time_col, 
        df_mp_removal_time_col=df_mp_removal_time_col, 
        df_mp_trsf_pole_nb_col=df_mp_trsf_pole_nb_col
    )
    #-------------------------
    return active_SNs_df

# ===============================
# END ACQUISITION METHODS
# ===============================

In [ ]:
def add_est_outg_times_to_axis(
    ax, 
    est_outg_times, 
    t_beg_kwargs=None,
    t_end_kwargs=None,
    include='both', 
    include_outage_limits_text=True
):
    r"""
    At vertical lines showing estimates outage times.
    
    Note: out_t_beg and out_t_end aren't really necessary, they simply direct which side of
          the estimated lines to print the text
    """
    #-------------------------
    assert(include in ['both', 'beg', 'end'])
    #-------------------------
    if t_beg_kwargs is None:
        t_beg_kwargs={}
    if t_end_kwargs is None:
        t_end_kwargs={}
    #-----
    dflt_t_beg_kwargs=dict(color='red', linestyle='--')
    dflt_t_end_kwargs=dict(color='green', linestyle='--')
    #-----
    t_beg_kwargs = Utilities.supplement_dict_with_default_values(to_supplmnt_dict=t_beg_kwargs, default_values_dict=dflt_t_beg_kwargs, inplace=False)
    t_end_kwargs = Utilities.supplement_dict_with_default_values(to_supplmnt_dict=t_end_kwargs, default_values_dict=dflt_t_end_kwargs, inplace=False)
    #-------------------------
    for dt_off_i, dt_on_i in est_outg_times:
        if include=='both' or include=='beg':
            ax.axvline(dt_off_i, **t_beg_kwargs)
        if include=='both' or include=='end':
            ax.axvline(dt_on_i, **t_end_kwargs)
        #-----
        if include_outage_limits_text:
            ax.text(dt_off_i, ax.get_ylim()[0], 'Est. outg. beg', 
                    rotation=90, verticalalignment='bottom', 
                    horizontalalignment='right' if dt_off_i<out_t_beg else 'left')
            ax.text(dt_on_i, ax.get_ylim()[0], 'Est. outg. end', 
                    rotation=90, verticalalignment='bottom', 
                    horizontalalignment='left' if dt_on_i>out_t_end else 'right')
    #-------------------------

# COMBINE AMI AND EDE

In [ ]:
def drop_multiindex_duplicate_idxs_for_csv_output(df):
    r"""
    Eliminate duplicate values from MultiIndex index before writing to CSV to make reading by eye easier.
    IMPORTANT!!!!! The CSV generated here isn't useful for futher manipulation using code, only for inspection by eye
                   Thus, it's intended more for when I deliver results to others, not for use myself
    -----
    Essentially, instead of the output looking like:
        idx0     idx1     col1   col2   col3
        a        1        ...    ...    ...     
        a        2        ...    ...    ...
        b        2        ...    ...    ...
        b        3        ...    ...    ...
    
    It will instead look like:
        idx0     idx1     col1   col2   col3
        a        1        ...    ...    ... 
                 2        ...    ...    ...
        b        2        ...    ...    ...
                 3        ...    ...    ...
    """
    #-------------------------
    # Make sure df has MultiIndex index, and each level has a name
    assert(df.index.nlevels>1)
    assert((len(df.index.names)==df.index.nlevels))
    assert(all([True if x else False for x in df.index.names]))
    idx_names = list(df.index.names)
    #-------------------------
    df = df.reset_index()
    #-------------------------
    dupl_srs_list = []
    for i_idx_lvl in range(len(idx_names)):
        dupl_srs_i = df.duplicated(subset=idx_names[:i_idx_lvl+1])
        dupl_srs_list.append(dupl_srs_i)
    assert(len(dupl_srs_list)==len(idx_names))
    #-------------------------
    for i_idx_lvl in range(len(idx_names)):
        df[idx_names[i_idx_lvl]] = df[idx_names[i_idx_lvl]].where(~dupl_srs_list[i_idx_lvl], '')
    #-------------------------
    return df